<a href="https://colab.research.google.com/github/Devision789/Captcha-Reader/blob/main/Captcha_Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Devision789***

**https://github.com/Devision789**

***ĐỌC MÃ CAPTCHA ***

lưu ý nếu các bạn chạy local thì xem xét trang github inference
yêu cầu bản python và cài đúng nếu bản python không phù hợp thì hệ thống sẽ báo lỗi no module name distubils ( có thể package khác mình quên tên nó rồi )
Mọi người cần hỗ trợ về vấn đề gì có thể liên hệ mình.


In [ ]:
!pip install inference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.7/485.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 940.0/940.0 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pandas

In [47]:
import os
import pandas as pd
import cv2
import numpy as np
from PIL import Image
from inference_sdk import InferenceHTTPClient
import base64

# Khởi tạo khách hàng inference
CLIENT = InferenceHTTPClient(
    api_url="https://infer.roboflow.com",
    api_key="qttWvOfq2j0uBMG7Xf0y" # thay bằng api của bạn
)

# Danh sách các ảnh cần xử lý
image_files = [
    "1.png", "2.png", "3.png", "4.png",
    "9.png", "6.png", "7.png", "8.png",
    "10.png"
]

# Đường dẫn tới file Excel
excel_file_path = "/content/ocr_base64.xlsx"

def preprocess_image(image_path):
    # Đọc ảnh
    img = cv2.imread(image_path)

    # Chuyển sang ảnh xám
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Áp dụng Gaussian blur để giảm nhiễu
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # Áp dụng adaptive thresholding
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Áp dụng morphological operations để làm sạch ảnh
    kernel = np.ones((2,2), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Chuyển đổi lại thành định dạng PIL Image
    pil_img = Image.fromarray(cleaned)

    # Lưu ảnh đã xử lý
    preprocessed_path = f"preprocessed_{os.path.basename(image_path)}"
    pil_img.save(preprocessed_path)

    return preprocessed_path

def postprocess_result(text):
    # Loại bỏ khoảng trắng
    text = text.strip().lower()

    # Loại bỏ các ký tự không phải chữ cái hoặc số
    cleaned_text = ''.join(c for c in text if c.isalnum())

    # Chuyển đổi 'O' thành '0' và 'I' thành '1' (thường gặp trong CAPTCHA)
    cleaned_text = cleaned_text.replace('O', '0').replace('I', '1')

    # Giới hạn độ dài (giả sử CAPTCHA có độ dài cố định là 6)
    return cleaned_text[:7]

def text_to_base64(text):
    # Chuyển đổi text thành dạng base64
    return base64.b64encode(text.encode('utf-8')).decode('utf-8')
# Hàm để phân tích kết quả OCR
def parse_ocr_result(result, image_name):
    raw_text = result.get("result", "")
    processed_text = postprocess_result(raw_text)
    base64_text = text_to_base64(processed_text)
    return {
        "Image Name": image_name,
        "Processed Result": processed_text,
        "Processed Result (Base64)": base64_text
    }


# Hàm để thêm dữ liệu vào file Excel
def append_to_excel(df, file_path):
    if os.path.exists(file_path):
        # Nếu file tồn tại, thêm dữ liệu vào file hiện có
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, df], ignore_index=True)
    else:
        # Nếu file không tồn tại, tạo một DataFrame mới
        updated_df = df

    # Lưu DataFrame đã cập nhật vào file Excel
    updated_df.to_excel(file_path, index=False)

# Danh sách để lưu các kết quả đã phân tích
all_data = []

# Lặp qua các ảnh và thực hiện OCR
for image_file in image_files:
    # Thực hiện OCR trên ảnh
    result = CLIENT.ocr_image(inference_input=image_file)



    # Phân tích kết quả OCR
    parsed_data = parse_ocr_result(result, os.path.basename(image_file))
    all_data.append(parsed_data)

# Tạo DataFrame từ tất cả các kết quả đã phân tích
df = pd.DataFrame(all_data)

# Thêm dữ liệu đã phân tích vào file Excel
append_to_excel(df, excel_file_path)

print(f"Dữ liệu đã được lưu vào {excel_file_path}")

Dữ liệu đã được lưu vào /content/ocr_base64.xlsx


In [46]:
import os
import pandas as pd
import cv2
import numpy as np
from PIL import Image
from inference_sdk import InferenceHTTPClient

# Khởi tạo khách hàng inference
CLIENT = InferenceHTTPClient(
    api_url="https://infer.roboflow.com",
    api_key="qttWvOfq2j0uBMG7Xf0y" # thay bằng api của bạn
)

# Danh sách các ảnh cần xử lý
image_files = [
    "1.png", "2.png", "3.png", "4.png",
    "9.png", "6.png", "7.png", "8.png",
    "10.png"
]

# Đường dẫn tới file Excel
excel_file_path = "/content/ocrs.xlsx"

def preprocess_image(image_path):
    # Đọc ảnh
    img = cv2.imread(image_path)

    # Chuyển sang ảnh xám
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Áp dụng Gaussian blur để giảm nhiễu
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # Áp dụng adaptive thresholding
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Áp dụng morphological operations để làm sạch ảnh
    kernel = np.ones((2,2), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Chuyển đổi lại thành định dạng PIL Image
    pil_img = Image.fromarray(cleaned)

    # Lưu ảnh đã xử lý
    preprocessed_path = f"preprocessed_{os.path.basename(image_path)}"
    pil_img.save(preprocessed_path)

    return preprocessed_path

def postprocess_result(text):
    # Loại bỏ khoảng trắng
    text = text.strip()

    # Loại bỏ các ký tự không phải chữ cái hoặc số
    cleaned_text = ''.join(c for c in text if c.isalnum())

    # Chuyển đổi 'O' thành '0' và 'I' thành '1' (thường gặp trong CAPTCHA)
    cleaned_text = cleaned_text.replace('O', '0').replace('I', '1')

    # Giới hạn độ dài (giả sử CAPTCHA có độ dài cố định là 6)
    return cleaned_text[:7]

# Hàm để phân tích kết quả OCR
def parse_ocr_result(result, image_name):
    raw_text = result.get("result", "")
    processed_text = postprocess_result(raw_text)
    return {
        "Image Name": image_name,
        "Processed Result": processed_text
    }

# Hàm để thêm dữ liệu vào file Excel
def append_to_excel(df, file_path):
    if os.path.exists(file_path):
        # Nếu file tồn tại, thêm dữ liệu vào file hiện có
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, df], ignore_index=True)
    else:
        # Nếu file không tồn tại, tạo một DataFrame mới
        updated_df = df

    # Lưu DataFrame đã cập nhật vào file Excel
    updated_df.to_excel(file_path, index=False)

# Danh sách để lưu các kết quả đã phân tích
all_data = []

# Lặp qua các ảnh và thực hiện OCR
for image_file in image_files:
    # Thực hiện OCR trên ảnh
    result = CLIENT.ocr_image(inference_input=image_file)

    # Phân tích kết quả OCR
    parsed_data = parse_ocr_result(result, os.path.basename(image_file))
    all_data.append(parsed_data)

# Tạo DataFrame từ tất cả các kết quả đã phân tích
df = pd.DataFrame(all_data)

# Thêm dữ liệu đã phân tích vào file Excel
append_to_excel(df, excel_file_path)


print(f"Dữ liệu đã được lưu vào {excel_file_path}")

Dữ liệu đã được lưu vào /content/ocrs.xlsx


In [ ]:
import base64

def text_to_base64(text):

    text_bytes = text.encode('utf-8')


    base64_bytes = base64.b64encode(text_bytes)


    base64_string = base64_bytes.decode('utf-8')

    return base64_string


text = "vqcc5j"
base64_result = text_to_base64(text)
print(f"Original text: {text}")
print(f"Base64 encoded: {base64_result}")

Original text: vqcc5j
Base64 encoded: dnFjYzVq


In [ ]:
import os
import pandas as pd
from inference_sdk import InferenceHTTPClient

# Khởi tạo khách hàng inference
CLIENT = InferenceHTTPClient(
    api_url="https://infer.roboflow.com",
    api_key="qttWvOfq2j0uBMG7Xf0y" # thay bằng api của bạn
)

# Danh sách các ảnh cần xử lý
image_files = [

    "1.png",
    "2.png",
    "3.png",
    "4.png",
    "5.png",
    "6.png",
    "7.png",
    "8.png"
]

# Đường dẫn tới file Excel
excel_file_path = "/content/ocrcaptcha.xlsx"

# Hàm để phân tích kết quả OCR
def parse_ocr_result(result, image_name):
    return {
        "Image Name": image_name,
        "Result": result.get("result", "")
    }

# Hàm để thêm dữ liệu vào file Excel
def append_to_excel(df, file_path):
    if os.path.exists(file_path):
        # Nếu file tồn tại, thêm dữ liệu vào file hiện có
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, df], ignore_index=True)
    else:
        # Nếu file không tồn tại, tạo một DataFrame mới
        updated_df = df

    # Lưu DataFrame đã cập nhật vào file Excel
    updated_df.to_excel(file_path, index=False)

# Danh sách để lưu các kết quả đã phân tích
all_data = []

# Lặp qua các ảnh và thực hiện OCR
for image_file in image_files:
    # Thực hiện OCR trên ảnh
    result = CLIENT.ocr_image(inference_input=image_file)

    # In kết quả
    print(result)

    # Phân tích kết quả OCR
    parsed_data = parse_ocr_result(result, os.path.basename(image_file))
    all_data.append(parsed_data)

# Tạo DataFrame từ tất cả các kết quả đã phân tích
df = pd.DataFrame(all_data)

# Thêm dữ liệu đã phân tích vào file Excel
append_to_excel(df, excel_file_path)

print(f"Dữ liệu đã được lưu vào {excel_file_path}")


{'result': 'vqcc5, J -', 'time': 1.5120850140001494, 'parent_id': None}
{'result': 'nddmdw', 'time': 1.5530674480000926, 'parent_id': None}
{'result': '3wyqc9', 'time': 1.4337150139999721, 'parent_id': None}
{'result': '6v6sj2', 'time': 1.5459657910000715, 'parent_id': None}
{'result': 'XWZ 46p', 'time': 1.512343518000307, 'parent_id': None}
{'result': '3f. /. Z 83', 'time': 1.4889416139999412, 'parent_id': None}
{'result': 't76rgt L', 'time': 1.4522427620004237, 'parent_id': None}
{'result': 'dmtrgz', 'time': 1.4663603539997894, 'parent_id': None}
Dữ liệu đã được lưu vào /content/ocrcaptcha.xlsx


In [ ]:
pip install python-doctr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 45.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
pip install "python-doctr[tf]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 48.8 MB/s eta 0:00:00
